In [ ]:
import paho.mqtt.client as mqttClient
import joblib
import tsfel
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

#Pilih topic yang ada pada simulasi TEP
topic = input("topic : ")  

class main():
    def __init__(self, topic):
        #Koneksi MQTT
        self.broker= "127.0.0.1"                                    #Broker address
        self.port = 1883                                            #Broker port
        self.client_id = f'python-mqtt-{random.randint(0, 100)}'    #client id 
        self.topic = topic
        self.data = []                                              #Data yang diterima dari TEP
        self.time = 0                                               #Perhitungan step akuisisi data
        
        #Model
        self.deteksi_model = joblib.load("Jendela 400.sav")           #Model ML deteksi 
        
        #self.deteksi_scaler = joblib.load("Deteksi_scaler")
        
        self.fitur_deteksi = ['0_Max power spectrum', '0_Maximum frequency', '0_Median frequency', '0_Power bandwidth', 
             '0_Spectral centroid', '0_Spectral decrease', '0_Spectral distance', '0_Spectral entropy', 
             '0_Spectral positive turning points', '0_Spectral skewness','0_Spectral spread', '0_Spectral variation', 
             '0_Wavelet absolute mean_0','0_Wavelet energy_3','0_Wavelet standard deviation_8','0_Histogram_9',
             '0_Interquartile range', '0_Kurtosis', '0_Max', '0_Mean', '0_Mean absolute deviation', '0_Median', 
             '0_Median absolute deviation', '0_Min', '0_Root mean square', '0_Skewness', '0_Standard deviation',
             '0_Entropy', '0_Mean absolute diff', '0_Mean diff', '0_Median absolute diff', '0_Median diff', 
             '0_Negative turning points', '0_Neighbourhood peaks', '0_Peak to peak distance', 
             '0_Positive turning points', '0_Signal distance', '0_Slope']
                       
        self.hasil_deteksi = ""
        

    def connect_mqtt(self): #Fungsi koneksi ke broker MQTT
        def on_connect(client, userdata, flags, rc):
            if rc == 0:
                    print("Subscriber Connected to MQTT Broker!")
            else:
                    print("Subscriber Failed to connect, return code %d\n", rc)

        global client_id, broker, port
        client = mqttClient.Client(self.client_id)
        client.on_connect = on_connect
        client.connect(self.broker, self.port)
        return client
    
    def on_message(self, client, userdata, message):    #Fungsi penerimaan pesan dari broker MQTT
        self.data.append(float(message.payload))        #Masukan data yang diterima ke dalam list
        self.time += 1                                  #Menghitung step penerimaan data 
        if len(self.data) > 400:    #Memulai Deteksi
            del self.data[0]
            data_awal = MinMaxScaler(feature_range=(0, 1)).fit_transform(np.array(self.data).reshape(-1,1))
            hasil = data_awal.reshape(1,-1)
            #Ekstrak TSFEL
            cgf_file = tsfel.get_features_by_domain()
            x_t = tsfel.time_series_features_extractor(cgf_file, hasil[0], fs=100,verbose=0)

            #Deteksi 
            X_det = x_t[self.fitur_deteksi]
            X_pred_det = pd.DataFrame(X_det)
            y_pred_det = self.deteksi_model.predict(X_pred_det)
            
            
            if y_pred_det == 0:
                self.hasil_deteksi = "Normal"
            elif y_pred_det == 1: 
                self.hasil_deteksi = "Stiction"
            elif y_pred_det == 2: 
                self.hasil_deteksi = "Stiction Mistuned"
            elif y_pred_det == 3: 
                self.hasil_deteksi = "Stiction External"
                    
            print("Time: ", self.time, "    PV: ", "{:.3f}".format(self.data[-1]), "    Detection Status: ", self.hasil_deteksi)
            #time.sleep(5) #digunakan untuk mengatur delay dari pengiriman data, bisa sesuaikan dengan resolusi sistem akuisisi data
        else:
            print("Time: ", self.time, "    PV: ", "{:.3f}".format(self.data[-1]), "    Detection Status: Not Started Yet     ")

    def run(self):
        client = self.connect_mqtt()
        client.subscribe(self.topic, qos = 1)
        client.on_message = self.on_message
        client.loop_forever()
        
    #df_mv1 = pd.DataFrame(mv1)
    #df_mv1.to_csv(r'C:/Users/Lenovo/Documents/TUGAS AKHIR/Dataset/IDV 14, TC 10, IDV 11/3. Osilasi Jamak/Kondisi 3/mv1.csv', header=False, index=False)

if __name__ == "__main__":
    main(topic).run()


Subscriber Connected to MQTT Broker!
Time:  1     PV:  94.599     Detection Status: Not Started Yet     
Time:  2     PV:  94.599     Detection Status: Not Started Yet     
Time:  3     PV:  94.584     Detection Status: Not Started Yet     
Time:  4     PV:  94.599     Detection Status: Not Started Yet     
Time:  5     PV:  94.587     Detection Status: Not Started Yet     
Time:  6     PV:  94.592     Detection Status: Not Started Yet     
Time:  7     PV:  94.601     Detection Status: Not Started Yet     
Time:  8     PV:  94.599     Detection Status: Not Started Yet     
Time:  9     PV:  94.618     Detection Status: Not Started Yet     
Time:  10     PV:  94.595     Detection Status: Not Started Yet     
Time:  11     PV:  94.596     Detection Status: Not Started Yet     
Time:  12     PV:  94.591     Detection Status: Not Started Yet     
Time:  13     PV:  94.599     Detection Status: Not Started Yet     
Time:  14     PV:  94.576     Detection Status: Not Started Yet     
Time: 

Time:  120     PV:  94.580     Detection Status: Not Started Yet     
Time:  121     PV:  94.582     Detection Status: Not Started Yet     
Time:  122     PV:  94.572     Detection Status: Not Started Yet     
Time:  123     PV:  94.587     Detection Status: Not Started Yet     
Time:  124     PV:  94.590     Detection Status: Not Started Yet     
Time:  125     PV:  94.570     Detection Status: Not Started Yet     
Time:  126     PV:  94.581     Detection Status: Not Started Yet     
Time:  127     PV:  94.579     Detection Status: Not Started Yet     
Time:  128     PV:  94.574     Detection Status: Not Started Yet     
Time:  129     PV:  94.555     Detection Status: Not Started Yet     
Time:  130     PV:  94.574     Detection Status: Not Started Yet     
Time:  131     PV:  94.568     Detection Status: Not Started Yet     
Time:  132     PV:  94.580     Detection Status: Not Started Yet     
Time:  133     PV:  94.579     Detection Status: Not Started Yet     
Time:  134     PV:  

Time:  238     PV:  94.600     Detection Status: Not Started Yet     
Time:  239     PV:  94.588     Detection Status: Not Started Yet     
Time:  240     PV:  94.585     Detection Status: Not Started Yet     
Time:  241     PV:  94.598     Detection Status: Not Started Yet     
Time:  242     PV:  94.589     Detection Status: Not Started Yet     
Time:  243     PV:  94.591     Detection Status: Not Started Yet     
Time:  244     PV:  94.596     Detection Status: Not Started Yet     
Time:  245     PV:  94.595     Detection Status: Not Started Yet     
Time:  246     PV:  94.599     Detection Status: Not Started Yet     
Time:  247     PV:  94.581     Detection Status: Not Started Yet     
Time:  248     PV:  94.579     Detection Status: Not Started Yet     
Time:  249     PV:  94.589     Detection Status: Not Started Yet     
Time:  250     PV:  94.577     Detection Status: Not Started Yet     
Time:  251     PV:  94.612     Detection Status: Not Started Yet     
Time:  252     PV:  

Time:  356     PV:  94.619     Detection Status: Not Started Yet     
Time:  357     PV:  94.611     Detection Status: Not Started Yet     
Time:  358     PV:  94.594     Detection Status: Not Started Yet     
Time:  359     PV:  94.607     Detection Status: Not Started Yet     
Time:  360     PV:  94.608     Detection Status: Not Started Yet     
Time:  361     PV:  94.592     Detection Status: Not Started Yet     
Time:  362     PV:  94.604     Detection Status: Not Started Yet     
Time:  363     PV:  94.593     Detection Status: Not Started Yet     
Time:  364     PV:  94.592     Detection Status: Not Started Yet     
Time:  365     PV:  94.575     Detection Status: Not Started Yet     
Time:  366     PV:  94.567     Detection Status: Not Started Yet     
Time:  367     PV:  94.582     Detection Status: Not Started Yet     
Time:  368     PV:  94.576     Detection Status: Not Started Yet     
Time:  369     PV:  94.577     Detection Status: Not Started Yet     
Time:  370     PV:  

Time:  490     PV:  94.480     Detection Status:  Normal
Time:  491     PV:  94.499     Detection Status:  Normal
Time:  492     PV:  94.483     Detection Status:  Normal
Time:  493     PV:  94.485     Detection Status:  Normal
Time:  494     PV:  94.512     Detection Status:  Normal
Time:  495     PV:  94.519     Detection Status:  Normal
Time:  496     PV:  94.523     Detection Status:  Normal
Time:  497     PV:  94.502     Detection Status:  Normal
Time:  498     PV:  94.509     Detection Status:  Normal
Time:  499     PV:  94.492     Detection Status:  Normal
Time:  500     PV:  94.489     Detection Status:  Normal
Time:  501     PV:  94.512     Detection Status:  Normal
Time:  502     PV:  94.524     Detection Status:  Normal
Time:  503     PV:  94.518     Detection Status:  Normal
Time:  504     PV:  94.515     Detection Status:  Normal
Time:  505     PV:  94.512     Detection Status:  Normal
Time:  506     PV:  94.534     Detection Status:  Normal
Time:  507     PV:  94.524     

...

Time:  12113     PV:  95.486     Detection Status:  Normal
Time:  12114     PV:  95.463     Detection Status:  Normal
Time:  12115     PV:  95.477     Detection Status:  Normal
Time:  12116     PV:  95.470     Detection Status:  Normal
Time:  12117     PV:  95.460     Detection Status:  Normal
Time:  12118     PV:  95.479     Detection Status:  Normal
Time:  12119     PV:  95.465     Detection Status:  Normal
Time:  12120     PV:  95.472     Detection Status:  Normal
Time:  12121     PV:  95.459     Detection Status:  Normal
Time:  12122     PV:  95.474     Detection Status:  Normal
Time:  12123     PV:  95.453     Detection Status:  Normal
Time:  12124     PV:  95.432     Detection Status:  Normal
Time:  12125     PV:  95.454     Detection Status:  Normal
Time:  12126     PV:  95.423     Detection Status:  Normal
Time:  12127     PV:  95.428     Detection Status:  Normal
Time:  12128     PV:  95.421     Detection Status:  Normal
Time:  12129     PV:  95.407     Detection Status:  Norm

Time:  12252     PV:  95.387     Detection Status:  Normal
Time:  12253     PV:  95.375     Detection Status:  Normal
Time:  12254     PV:  95.383     Detection Status:  Normal
Time:  12255     PV:  95.372     Detection Status:  Normal
Time:  12256     PV:  95.349     Detection Status:  Normal
Time:  12257     PV:  95.364     Detection Status:  Normal
Time:  12258     PV:  95.356     Detection Status:  Normal
Time:  12259     PV:  95.335     Detection Status:  Normal
Time:  12260     PV:  95.350     Detection Status:  Normal
Time:  12261     PV:  95.345     Detection Status:  Normal
Time:  12262     PV:  95.355     Detection Status:  Normal
Time:  12263     PV:  95.354     Detection Status:  Normal
Time:  12264     PV:  95.343     Detection Status:  Normal
Time:  12265     PV:  95.332     Detection Status:  Normal
Time:  12266     PV:  95.353     Detection Status:  Normal
Time:  12267     PV:  95.324     Detection Status:  Normal
Time:  12268     PV:  95.356     Detection Status:  Norm

Time:  12391     PV:  95.282     Detection Status:  Normal
Time:  12392     PV:  95.304     Detection Status:  Normal
Time:  12393     PV:  95.304     Detection Status:  Normal
Time:  12394     PV:  95.303     Detection Status:  Normal
Time:  12395     PV:  95.296     Detection Status:  Normal
Time:  12396     PV:  95.306     Detection Status:  Normal
Time:  12397     PV:  95.285     Detection Status:  Normal
Time:  12398     PV:  95.289     Detection Status:  Normal
Time:  12399     PV:  95.294     Detection Status:  Normal
Time:  12400     PV:  95.309     Detection Status:  Normal
Time:  12401     PV:  95.311     Detection Status:  Normal
Time:  12402     PV:  95.316     Detection Status:  Normal
Time:  12403     PV:  95.304     Detection Status:  Normal
Time:  12404     PV:  95.331     Detection Status:  Normal
Time:  12405     PV:  95.315     Detection Status:  Normal
Time:  12406     PV:  95.311     Detection Status:  Normal
Time:  12407     PV:  95.326     Detection Status:  Norm

Time:  12530     PV:  95.255     Detection Status:  Normal
Time:  12531     PV:  95.274     Detection Status:  Normal
Time:  12532     PV:  95.278     Detection Status:  Normal
Time:  12533     PV:  95.258     Detection Status:  Normal
Time:  12534     PV:  95.274     Detection Status:  Normal
Time:  12535     PV:  95.266     Detection Status:  Normal
Time:  12536     PV:  95.286     Detection Status:  Normal
Time:  12537     PV:  95.283     Detection Status:  Normal
Time:  12538     PV:  95.283     Detection Status:  Normal
Time:  12539     PV:  95.298     Detection Status:  Normal
Time:  12540     PV:  95.298     Detection Status:  Normal
Time:  12541     PV:  95.299     Detection Status:  Normal
Time:  12542     PV:  95.299     Detection Status:  Normal
Time:  12543     PV:  95.312     Detection Status:  Normal
Time:  12544     PV:  95.323     Detection Status:  Normal
Time:  12545     PV:  95.325     Detection Status:  Normal
Time:  12546     PV:  95.333     Detection Status:  Norm

Time:  12669     PV:  95.270     Detection Status:  Normal
Time:  12670     PV:  95.253     Detection Status:  Normal
Time:  12671     PV:  95.275     Detection Status:  Normal
Time:  12672     PV:  95.256     Detection Status:  Normal
Time:  12673     PV:  95.270     Detection Status:  Normal
Time:  12674     PV:  95.271     Detection Status:  Normal
Time:  12675     PV:  95.282     Detection Status:  Normal
Time:  12676     PV:  95.280     Detection Status:  Normal
Time:  12677     PV:  95.269     Detection Status:  Normal
Time:  12678     PV:  95.261     Detection Status:  Normal
Time:  12679     PV:  95.275     Detection Status:  Normal
Time:  12680     PV:  95.269     Detection Status:  Normal
Time:  12681     PV:  95.267     Detection Status:  Normal
Time:  12682     PV:  95.286     Detection Status:  Normal
Time:  12683     PV:  95.268     Detection Status:  Normal
Time:  12684     PV:  95.279     Detection Status:  Normal
Time:  12685     PV:  95.280     Detection Status:  Norm